In [8]:
from __future__ import print_function
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import pandas as pd


SPREADSHEET_ID = '101S-DEzHcZYNDw1VZ7NRXD2nkJQfq2TT_zvH9_1zcfU'
RANGE_NAME = 'list_1'

def get_google_sheet(spreadsheet_id, range_name):
    """ Retrieve sheet data using OAuth credentials and Google Python API. """
    scopes = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    # Setup the Sheets API
    store = file.Storage('credentials.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('client_secret.json', scopes)
        creds = tools.run_flow(flow, store)
    service = build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    gsheet = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    return gsheet


def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet.get('values', [])[0]   # Assumes first line is header!
    values = gsheet.get('values', [])[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df


gsheet = get_google_sheet(SPREADSHEET_ID, RANGE_NAME)
df = gsheet2df(gsheet)
print('Dataframe size = ', df.shape)
print(df)

Dataframe size =  (78, 4)
                 word T/F   cn_meaning  \
0             divulge   F           泄漏   
1        inadvertent    F     粗心的，不留意的   
2                   /             不故意的   
3           panegyric   F        赞文；赞美   
4          draconian    F        极其严苛的   
5         hodgepodge    F       混杂，混合物   
6   incontrovertible    F        不容置疑的   
7          impartial    T    公正的，不偏不倚的   
8             endemic   F     地方性的，流行的   
9                   /       （问题，情形）常见的   
10         minuscule    F          微小的   
11          lucrative   F      盈利的，赚钱的   
12         compensate   T           弥补   
13                  /               抵消   
14         discursive   F        杂乱无章的   
15                  /              理性的   
16         presuppose   T         预先假设   
17                  /               假定   
18       intermediary   F   （作为）中间人（的）   
19           derogate   F           贬低   
20        overbearing   F        专横傲慢的   
21            zealous   T  对（某人或偶像）狂热的   
22      